In [ ]:
### TODO Please provide your Cloudant credentials in this cell

#Please don't modify this function
def readDataFrameFromCloudant(database):
    cloudantdata=spark.read.load(database, "com.cloudant.spark")
    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",'XXXXX')\
    .config("cloudant.username", 'XXXXX')\
    .config("cloudant.password",'XXXXX')\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

In [ ]:
df=readDataFrameFromCloudant('training')

In [ ]:
# Enable SQL on the data frame
df.createOrReplaceTempView('df')

In [ ]:
df_class_0 = spark.sql('select time, temp, humidity, class from df where class = 0')
df_class_1 = spark.sql('select time, temp, humidity, class from df where class = 1')
df_class_0.createOrReplaceTempView('df_class_0')
df_class_1.createOrReplaceTempView('df_class_1')

In [ ]:
df_class_1.select('temp', 'humidity').distinct().show()

In [ ]:
df_class_0.select('temp', 'humidity').distinct().show()

In [ ]:
df.printSchema()

In [ ]:
spark.sql('select class, count(class) from df group by class').show()

In [ ]:
import pixiedust

In [ ]:
display(df_class_0)

In [ ]:
# Imports for modelling
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# create binary classifier model
vectorAssembler = VectorAssembler(inputCols=["humidity","temp"],
                                  outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df)
result = model.transform(df)

In [ ]:
model.stages[1].coefficients

In [ ]:
model.stages[1].intercept

In [ ]:
#evaluate classification accuracy (1.0 = 100% accurate)
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")    
binEval.evaluate(result)

In [ ]:
# test the model
#re-read data from cloudant
new_df = readDataFrameFromCloudant('training')
result = model.transform(new_df)
result.createOrReplaceTempView('result')
spark.sql("select humidity, temp, class, prediction from result").show(50)